In [8]:
pip install slideio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!apt-get install openslide-tools
!apt-get install python-openslide
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python-openslide
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
import requests
import slideio
import matplotlib.pyplot as plt
from PIL import Image
import os
from openslide import open_slide
import openslide
from PIL import Image
import numpy as np

## Resmi Tile'a dönüştürme

In [12]:
from openslide.deepzoom import DeepZoomGenerator

In [13]:
slide = open_slide('/content/gdrive/MyDrive/FinalProject/project/TCGA-SKCM/TCGA-EB-A1NK-01A-01-TSA.1A22E8A9-5F6B-4581-BB2A-1F63D0AA29C2.svs')
print(slide)

tile_dir = "/content/gdrive/MyDrive/comp411project/FinalProjectForTiles/TCGA-SKCM/tiles/"


tile_name = os.path.join(tile_dir, 'deneme' + '%d_%d' % (0, 0))
          

tile_name = os.path.join(tile_dir, 'deneme' + '%d_%d' % (0, 0))


tiles = DeepZoomGenerator(slide, tile_size=1024, overlap=0, limit_bounds=False)

print(tile_name)

OpenSlide('/content/gdrive/MyDrive/FinalProject/project/TCGA-SKCM/TCGA-EB-A1NK-01A-01-TSA.1A22E8A9-5F6B-4581-BB2A-1F63D0AA29C2.svs')
/content/gdrive/MyDrive/comp411project/FinalProjectForTiles/TCGA-SKCM/tiles/deneme0_0


In [14]:
print(tiles)

DeepZoomGenerator(OpenSlide('/content/gdrive/MyDrive/FinalProject/project/TCGA-SKCM/TCGA-EB-A1NK-01A-01-TSA.1A22E8A9-5F6B-4581-BB2A-1F63D0AA29C2.svs'), tile_size=1024, overlap=0, limit_bounds=False)


In [44]:
from PIL import Image
import traceback
import tifffile as tiff
from tifffile import imsave
from resizeimage import resizeimage
import cv2
import PIL

def generate_tiles(path, tile_dir) :

  svs_list = [svs_file for svs_file in os.listdir(path) if svs_file.endswith(".svs")]


  for s in svs_list:

    try:

      #Load the slide file (svs) into an object.

      print('Path', path+s)
      slide = open_slide(path + s)
          
      name = s[:-4]
      name = name + "_"

      tiles = DeepZoomGenerator(slide, tile_size=1024, overlap=0, limit_bounds=False)
      #print("The number of levels in the tiles object are: ", tiles.level_count)
      #print("The dimensions of data in each level are: ", tiles.level_dimensions)

      #Total number of tiles in the tiles object
      print("Total number of tiles = : ", tiles.tile_count)

      #How many tiles at a specific level?
      level_num = 12
      print("Tiles shape at level ", level_num, " is: ", tiles.level_tiles[level_num])
      print("This means there are ", tiles.level_tiles[level_num][0]*tiles.level_tiles[level_num][1], " total tiles in this level")

      ###### Saving each tile to local directory
      cols, rows = tiles.level_tiles[12]
      
      for row in range(rows):
        for col in range(cols):
          
          tile_name = os.path.join(tile_dir, name + '%d_%d' % (col, row))
        
          temp_tile = tiles.get_tile(12, (col, row))
          print('Get Tile:', type(temp_tile))
          temp_tile_array = np.array(temp_tile)
          temp_tile = cv2.resize(temp_tile_array, (224,224))
          print(type(temp_tile))
          temp_tile_final = PIL.Image.fromarray(temp_tile)
          temp_tile_RGB = temp_tile_final.convert('RGB')
          temp_tile_np = np.array(temp_tile_RGB)
          #plt.imsave(tile_name + ".tiff", temp_tile_np)    
          
          # a good image
          if temp_tile_np.mean() < 200 and temp_tile_np.std() > 20: 
            print("Now saving tile with title: ", tile_name)
            print(temp_tile_np.shape)
            #temp_tile_np = resizeimage.resize_cover(temp_tile_np, [224,224])
            #temp_tile_np.save(tile_name + ".tiff", temp_tile_np.format)
            imsave(tile_name + ".tiff", temp_tile_np) 

      
    except:
      print('Exception oluştu')
      traceback.print_exc()
      break
  return



In [ ]:
s_path = '/content/gdrive/MyDrive/FinalProject/project/TCGA-SKCM/'
tile_dir = "/content/gdrive/MyDrive/FinalProject/FinalProjectForTiles/TCGA-SKCM/tiles/"

generate_tiles(s_path,tile_dir)



Path /content/gdrive/MyDrive/FinalProject/project/TCGA-SKCM/TCGA-EB-A1NK-01A-01-TSA.1A22E8A9-5F6B-4581-BB2A-1F63D0AA29C2.svs
Total number of tiles = :  2910
Tiles shape at level  12  is:  (3, 1)
This means there are  3  total tiles in this level
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Path /content/gdrive/MyDrive/FinalProject/project/TCGA-SKCM/TCGA-EE-A2MC-01Z-00-DX1.F91A5A23-F552-4D34-9848-F95892E1B2C9.svs
Total number of tiles = :  10555
Tiles shape at level  12  is:  (4, 3)
This means there are  12  total tiles in this level
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Imag

<ipython-input-44-dca0005eab4a>:62: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(tile_name + ".tiff", temp_tile_np)


Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Now saving tile with title:  /content/gdrive/MyDrive/FinalProject/FinalProjectForTiles/TCGA-SKCM/tiles/TCGA-EE-A2MC-01Z-00-DX1.F91A5A23-F552-4D34-9848-F95892E1B2C9_2_1
(224, 224, 3)
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Path /content/gdrive/MyDrive/FinalProject/project/TCGA-SKCM/TCGA-EE-A2MC-06A-01-TSA.53AB0AA8-DE6F-4DFD-BC05-542A3051A5D3.svs
Total number of tiles = :  2542
Tiles shape at level  12  is:  (3, 1)
This means there are  3  total tiles in this level
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Get Tile: <class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
Path /content/gdrive/MyDrive/Fin

## Not

In [ ]:
s_path = '/content/gdrive/MyDrive/comp411project/FinalProject/project/TCGA-LUSC/'
tile_dir = "/content/gdrive/MyDrive/comp411project/FinalProjectForTiles/TCGA-LUSC/tiles/"

generate_tiles(s_path,tile_dir)
s_path = '/content/gdrive/MyDrive/comp411project/FinalProject/project/TCGA-COAD/'
tile_dir = "/content/gdrive/MyDrive/comp411project/FinalProjectForTiles/TCGA-COAD/tiles/"

generate_tiles(s_path,tile_dir)